In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [2]:
!pip install streamlit-mic-recorder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 11.8 MB/s eta 0:00:00


In [3]:
!pip install keras

In [8]:
%%writefile app.py

import streamlit as st
import librosa
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model as model_loader
import joblib
from streamlit_mic_recorder import mic_recorder
import wave


audio_file = None
model = None

def load_model():
    try:
        model = model_loader("CRNNSpoof_Model.h5")
        return model
    except FileNotFoundError:
        st.sidebar.error("Model file not found. Please make sure the file exists.")
        return None

model = load_model()
st.sidebar.success("Model loaded!")


st.title("Voice authentication 🎵:")

audio_file = st.file_uploader("Upload Audio", type=["wav","mp3","flac"])


def callback():
    if st.session_state.my_recorder_output:
        audio_bytes = st.session_state.my_recorder_output['bytes']
        st.audio(audio_bytes)


audio_file_test = mic_recorder(key='my_recorder', callback=callback, format = 'wav')
if audio_file_test is not None:
    with wave.open('output.wav', 'wb') as wav_file:
            audio_bytes = st.session_state.my_recorder_output['bytes']
            wav_file.setnchannels(1) # Mono
            wav_file.setsampwidth(2) # Sample width in bytes
            wav_file.setframerate(44100) # Sample rate
            wav_file.writeframes(audio_bytes)


st.sidebar.header("Play the audio")
st.sidebar.audio(audio_file)


def extract_features(file_path, segment_length):
    try:
        # Load the audio file
        y, sr = librosa.load(file_path)
        num_segments = int(np.ceil(len(y) / float(segment_length * sr)))
        print("num_segments: "+ str(num_segments))
        features = []

        for i in range(num_segments):
            start_frame = i * segment_length * sr
            end_frame = min(len(y), (i + 1) * segment_length * sr)
            y_segment = y[start_frame:end_frame]

            chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_segment, sr=sr))
            rms = np.mean(librosa.feature.rms(y=y_segment))
            spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_segment, sr=sr))
            spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_segment, sr=sr))
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_segment, sr=sr))
            zcr = np.mean(librosa.feature.zero_crossing_rate(y_segment))
            mfccs = librosa.feature.mfcc(y=y_segment, sr=sr)
            mfccs_mean = np.mean(mfccs, axis=1)

            features.append([chroma_stft, rms, spec_cent, spec_bw, rolloff, zcr, *mfccs_mean])

        return np.array(features)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


def make_prediction(features):
    output = model.predict(features)
    return [1 if value >= 0.5 else 0 for value in output]


if st.sidebar.button("Test Audio"):
  if audio_file is not None or os.path.isfile("./output.wav"):
    if os.path.isfile("./output.wav"):
      audio_file = "./output.wav"
    st.sidebar.success("Processing audio...")

    features = extract_features(audio_file, segment_length=1)
    X = pd.DataFrame(features, columns = ['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'])
    scaler = joblib.load("CRNNSpoof_scale.joblib")

    X_test_scaled = scaler.transform(features.reshape(features.shape[0], -1))
    X_test_scaled = X_test_scaled.reshape(features.shape)

    X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


    st.write("Making predictions with the model...")

    if features is not None:
        prediction_result = make_prediction(X_test_scaled)

    if np.average(prediction_result) <= 0.4:
      st.write("Prediction: Fake ")
    else:
      st.write("Prediction: Real ")
    if os.path.isfile("./output.wav"):
      os.remove("./output.wav")
  else:
    st.sidebar.warning("No Model")

Writing app.py


In [9]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.483s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [10]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.32.214.242


In [11]:
!streamlit run app.py &>/content/logs.txt &

In [12]:
!npx localtunnel --port 8501

npx: installed 22 in 1.25s
your url is: https://heavy-mails-grow.loca.lt
^C
